In [ ]:
pip install wget

In [1]:
import astropy
from astropy.table import Table,vstack, Column
from subprocess import call
import subprocess
from astropy.io import fits
import numpy as np
import wget
import matplotlib.pyplot as plt
from gpz_pype import utilities
from gpz_pype.utilities import Params, set_gpz_path, basic_lupt_soft, flux_to_lupt
import astropy.units as u
from numpy.ma import masked
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky

from os import listdir
from os.path import isfile, join

In [ ]:
#['ceers-full','gdn','primer-uds-south'
fields= ['gds','ngdeep','primer-cosmos-east','primer-cosmos-west','primer-uds-north','abell2744clu','abells1063','elgordo','j112716p4228','macs0647','rxj2129','smacs0723','sunrise']

In [35]:
flux = Table.read('MPhys/original_jwst_data/all_flux_merged.fits',format='fits')

FileNotFoundError: [Errno 2] No such file or directory: 'MPhys/original_jwst_data/all_flux_merged.csv'

In [13]:
for i in range(len(flux.colnames)):
    print(i,flux.colnames[i])

0 field
1 id
2 thresh
3 npix
4 tnpix
5 xmin
6 xmax
7 ymin
8 ymax
9 x
10 y
11 x2_image
12 y2_image
13 xy_image
14 errx2
15 erry2
16 errxy
17 a_image
18 b_image
19 theta_image
20 cxx_image
21 cyy_image
22 cxy_image
23 cflux
24 flux
25 cpeak
26 peak
27 xcpeak
28 ycpeak
29 xpeak
30 ypeak
31 flag
32 x_image
33 y_image
34 number
35 ra
36 dec
37 x_world
38 y_world
39 flux_iso
40 fluxerr_iso
41 area_iso
42 mag_iso
43 kron_radius
44 kron_rcirc
45 flux_auto
46 fluxerr_auto
47 bkg_auto
48 flag_auto
49 area_auto
50 flux_radius_flag
51 flux_radius_20
52 flux_radius
53 flux_radius_90
54 tot_corr
55 mag_auto
56 magerr_auto
57 flux_aper_0
58 fluxerr_aper_0
59 flag_aper_0
60 bkg_aper_0
61 mask_aper_0
62 flux_aper_1
63 fluxerr_aper_1
64 flag_aper_1
65 bkg_aper_1
66 mask_aper_1
67 flux_aper_2
68 fluxerr_aper_2
69 flag_aper_2
70 bkg_aper_2
71 mask_aper_2
72 flux_aper_3
73 fluxerr_aper_3
74 flag_aper_3
75 bkg_aper_3
76 mask_aper_3
77 f1000w_flux_aper_0
78 f1000w_fluxerr_aper_0
79 f1000w_flag_aper_0
80 f100

497 f560w_flux_aper_0
498 f560w_fluxerr_aper_0
499 f560w_flag_aper_0
500 f560w_bkg_aper_0
501 f560w_mask_aper_0
502 f560w_flux_aper_1
503 f560w_fluxerr_aper_1
504 f560w_flag_aper_1
505 f560w_bkg_aper_1
506 f560w_mask_aper_1
507 f560w_flux_aper_2
508 f560w_fluxerr_aper_2
509 f560w_flag_aper_2
510 f560w_bkg_aper_2
511 f560w_mask_aper_2
512 f560w_flux_aper_3
513 f560w_fluxerr_aper_3
514 f560w_flag_aper_3
515 f560w_bkg_aper_3
516 f560w_mask_aper_3
517 f560w_tot_corr
518 f606w_flux_aper_0
519 f606w_fluxerr_aper_0
520 f606w_flag_aper_0
521 f606w_bkg_aper_0
522 f606w_mask_aper_0
523 f606w_flux_aper_1
524 f606w_fluxerr_aper_1
525 f606w_flag_aper_1
526 f606w_bkg_aper_1
527 f606w_mask_aper_1
528 f606w_flux_aper_2
529 f606w_fluxerr_aper_2
530 f606w_flag_aper_2
531 f606w_bkg_aper_2
532 f606w_mask_aper_2
533 f606w_flux_aper_3
534 f606w_fluxerr_aper_3
535 f606w_flag_aper_3
536 f606w_bkg_aper_3
537 f606w_mask_aper_3
538 f606w_tot_corr
539 f606wu_flux_aper_0
540 f606wu_fluxerr_aper_0
541 f606wu_flag_a

In [31]:
#for i in range(len(flux.colnames)):
    #print(i,flux.colnames[i])
keep_cols = flux.colnames[0:2]+flux.colnames[35:37]+flux.colnames[62:len(flux.colnames)+1]
keeps=['field', 'id', 'ra', 'dec','flux_auto']
for i in range(4,len(keep_cols)):
    if keep_cols[i].startswith('flux'):
        if keep_cols[i].endswith('aper_1') or keep_cols[i].endswith('aper_2'):
            keeps.append(keep_cols[i])
    elif keep_cols[i].endswith('flux_aper_1') or keep_cols[i].endswith('flux_aper_2') or keep_cols[i].endswith('fluxerr_aper_1') or keep_cols[i].endswith('fluxerr_aper_2'):
        keeps.append(keep_cols[i])


In [32]:
cut_flux = flux[keeps]

In [36]:
cut_flux.write('MPhys/original_jwst_data/cut_flux_photometry_only.csv',format='csv')

# Download Field Files


In [ ]:
#for field in fields:
url=f'https://s3.amazonaws.com/grizli-v2/JwstMosaics/v7/gdn-grizli-v7.0_phot.fits'
filename=wget.download(url)
#print(f' {field} downloaded')

In [ ]:
hdul= fits.open('MPhys/DAWN_JWST/ceers-full-grizli-v7.0_phot.fits', memmap=True,formats='fits')
t = Table.read(hdul)
t

# Initial Catalogue Making - Run Once


In [ ]:

###INITIAL CATALOGUE MAKING
hdu_list = fits.open('ceers-full-grizli-v7.0_phot.fits', memmap=True,formats='fits')
ceers = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('gdn-grizli-v7.0_phot.fits', memmap=True,formats='fits')
gdn= Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('gds-grizli-v7.0_phot.fits', memmap=True,formats='fits')
gds = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('ngdeep-grizli-v7.0_phot.fits', memmap=True,formats='fits')
ngdeep = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('primer-cosmos-east-grizli-v7.0_phot.fits', memmap=True,formats='fits')
cosmos_east = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('primer-cosmos-west-grizli-v7.0_phot.fits', memmap=True,formats='fits')
cosmos_west = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('primer-uds-south-grizli-v7.0_phot.fits', memmap=True,formats='fits')
udss = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('primer-uds-north-grizli-v7.0_phot.fits', memmap=True,formats='fits')
udsn = Table(hdu_list[1].data)
hdu_list.close()

In [ ]:
#ADDING FIELDS TO CATALOGUE - RUN BOTH CELLS IF STARTING FROM SCRATCH
hdu_list = fits.open('abell2744clu-grizli-v7.0_phot.fits', memmap=True,formats='fits')
abell2744 = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('abells1063-grizli-v7.0_phot.fits', memmap=True,formats='fits')
abells1063= Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('elgordo-grizli-v7.0_phot.fits', memmap=True,formats='fits')
elgordo = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('j112716p4228-grizli-v7.0_phot.fits', memmap=True,formats='fits')
j_field = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('macs0647-grizli-v7.0_phot.fits', memmap=True,formats='fits')
macs = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('rxj2129-grizli-v7.0_phot.fits', memmap=True,formats='fits')
rxj = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('smacs0723-grizli-v7.0_phot.fits', memmap=True,formats='fits')
smacs = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('sunrise-grizli-v7.0_phot.fits', memmap=True,formats='fits')
sunrise = Table(hdu_list[1].data)
hdu_list.close()

In [ ]:
abell2744.add_column('abells2744clu', name='field', index=0)
abells1063.add_column('abells1063', name='field', index=0)
elgordo.add_column('elgordo', name='field', index=0)
j_field.add_column('j112716p4228', name='field', index=0)
macs.add_column('macs0647', name='field', index=0)
rxj.add_column('rxj2129', name='field', index=0)
smacs.add_column('smacs0723', name='field', index=0)
sunrise.add_column('sunrise', name='field', index=0)

In [ ]:
ceers.add_column('ceers', name='field', index=0)
gdn.add_column('gdn', name='field', index=0)
gds.add_column('gds', name='field', index=0)
ngdeep.add_column('ngdeep', name='field', index=0)
cosmos_east.add_column('cosmos_east', name='field', index=0)
cosmos_west.add_column('cosmos_west', name='field', index=0)
udsn.add_column('udsn', name='field', index=0)
udss.add_column('udss', name='field', index=0)

In [ ]:
cosmos_east

In [ ]:
main = vstack([ceers,gds,gdn,ngdeep,udss,udsn,cosmos_east,cosmos_west])

In [ ]:
m2 = vstack([main,ngdeep,cosmos_east])#,cosmos_west])

In [ ]:
m3 = vstack([m2,udsn,udss])

In [ ]:
m4 = vstack([m3,cosmos_west])

In [ ]:
side = vstack([abell2744,abells1063,elgordo,j_field,macs,rxj,smacs,sunrise])

In [ ]:
main.write('MPhys/original_jwst_data/main_merged.fits',format='fits',overwrite=True)

In [ ]:
side.write('MPhys/original_jwst_data/other_merged.fits',format='fits',overwrite=True)

In [ ]:
#all_merge = vstack([main,side])

In [ ]:
#all_merge.write('MPhys/original_jwst_data/all_merged_original.fits',format='fits',overwrite=True)

In [ ]:
#fields_new= ['ceers','gdn','gds','ngdeep','cosmos_east','cosmos_west','udsn','udss','abell2744','abells1063','elgordo','j_field','macs','rxj','smacs','sunrise']

In [ ]:
jwst_merge = Table.read('MPhys/original_jwst_data/all_flux_merged.fits',format ='fits')

In [ ]:
jwst_merge.add_column(np.nan,name = 'z_spec',index = 4)
jwst_merge.write('MPhys/original_jwst_data/all_merged_ab.fits',format='fits',overwrite=True)

In [ ]:
jwst_merge

## CANDELS

In [ ]:
cosmos = Table.read('MPhys/CANDELS/CANDELS.COSMOS.F160W.Processed.photz.fits',format='fits')
egs = Table.read('MPhys/CANDELS/CANDELS.EGS.F160W.Processed.photz.fits',format='fits')
uds_candels = Table.read('MPhys/CANDELS/CANDELS.UDS.F160W.Processed.photz.fits',format='fits')
gds_candels = Table.read('MPhys/CANDELS/CANDELS.GOODSS.F160W.Processed.photz.fits',format='fits')
gdn_candels = Table.read('MPhys/CANDELS/CANDELS.GOODSN.F160W.Processed.photz.fits',format='fits')

In [ ]:
candels = vstack([cosmos,egs,uds_candels,gds_candels,gdn_candels])

In [ ]:
candels.write('MPhys/CANDELS/candels_combined.fits',format='fits')

# Convert to Luptiudes for Each Field


In [ ]:
field=gds


In [ ]:
filters0 = [col.split('_')[0] for col in field.colnames if col.endswith('flux_aper_0')]
filters0=filters0[1:]
filters1 = [col.split('_')[0] for col in field.colnames if col.endswith('flux_aper_1')]
filters1=filters1[1:]

filters2 = [col.split('_')[0] for col in field.colnames if col.endswith('flux_aper_2')]
filters2=filters2[1:]

filters3 = [col.split('_')[0] for col in field.colnames if col.endswith('flux_aper_3')]
filters3=filters3[1:]



In [ ]:

b_arr0 = [basic_lupt_soft(field[f'{filt}_flux_aper_0'], field[f'{filt}_fluxerr_aper_0']) for filt in filters0] 
b_arr1 = [basic_lupt_soft(field[f'{filt}_flux_aper_1'], field[f'{filt}_fluxerr_aper_1']) for filt in filters1] 
b_arr2 = [basic_lupt_soft(field[f'{filt}_flux_aper_2'], field[f'{filt}_fluxerr_aper_2']) for filt in filters2] 
b_arr3 = [basic_lupt_soft(field[f'{filt}_flux_aper_3'], field[f'{filt}_fluxerr_aper_3']) for filt in filters3] 

In [ ]:

UnitTypeError: Longitude instances require units equivalent to 'rad', bui=2
lupt_cols=field['field','id','ra','dec']

for filt,b in zip(filters0,b_arr0):
    lupt,lupterr = flux_to_lupt(field[f'{filt}_flux_aper_0'],field[f'{filt}_fluxerr_aper_0'],b)
    lupt_cols[f'lupt_{filt}_aper0']=lupt
    lupt_cols[f'lupterr_{filt}_aper0']=lupterr
    
for filt,b in zip(filters1,b_arr1):
    lupt,lupterr = flux_to_lupt(field[f'{filt}_flux_aper_1'],field[f'{filt}_fluxerr_aper_1'],b)
    lupt_cols[f'lupt_{filt}_aper1']=lupt
    lupt_cols[f'lupterr_{filt}_aper1']=lupterr
for filt,b in zip(filters2,b_arr2):
    lupt,lupterr = flux_to_lupt(field[f'{filt}_flux_aper_2'],field[f'{filt}_fluxerr_aper_2'],b)
    lupt_cols[f'lupt_{filt}_aper2']=lupt
    lupt_cols[f'lupterr_{filt}_aper2']=lupterr
for filt,b in zip(filters3,b_arr3):
    lupt,lupterr = flux_to_lupt(field[f'{filt}_flux_aper_3'],field[f'{filt}_fluxerr_aper_3'],b)
    lupt_cols[f'lupt_{filt}_aper3']=lupt
    lupt_cols[f'lupterr_{filt}_aper3']=lupterr

lupt_cols.write(f'MPhys/catalogues/{fields[i]}_lupts.fits',format='fits', overwrite=True)

In [ ]:
t = Table.read('MPhys/catalogues/gds_lupts.fits',format ='fits')
t

In [ ]:
lup_ceers=Table.read('MPhys/catalogues/ceers-full_lupts.fits',format='fits')
lup_gds=Table.read('MPhys/catalogues/gds_lupts.fits',format='fits')
lup_gdn=Table.read('MPhys/catalogues/gdn_lupts.fits',format='fits')
lup_ngdeep=Table.read('MPhys/catalogues/ngdeep_lupts.fits',format='fits')
lup_udss=Table.read('MPhys/catalogues/primer-uds-south_lupts.fits',format='fits')
lup_udsn=Table.read('MPhys/catalogues/primer-uds-north_lupts.fits',format='fits')
lup_cosmos_east=Table.read('MPhys/catalogues/primer-cosmos-east_lupts.fits',format='fits')
lup_cosmos_west=Table.read('MPhys/catalogues/primer-cosmos-west_lupts.fits',format='fits')

In [ ]:
lup_abell2744clu = Table.read('MPhys/catalogues/abell2744clu_lupts.fits',format='fits')
lup_abells1063=Table.read('MPhys/catalogues/abells1063_lupts.fits',format='fits')
lup_elgordo=Table.read('MPhys/catalogues/elgordo_lupts.fits',format='fits')
lup_j112716p4228=Table.read('MPhys/catalogues/j112716p4228_lupts.fits',format='fits')
lup_macs0647=Table.read('MPhys/catalogues/macs0647_lupts.fits',format='fits')
lup_rxj2129=Table.read('MPhys/catalogues/rxj2129_lupts.fits',format='fits')
lup_smacs0723=Table.read('MPhys/catalogues/smacs0723_lupts.fits',format='fits')
lup_sunrise=Table.read('MPhys/catalogues/sunrise_lupts.fits',format='fits')

In [ ]:
#saves laptop dying if split up 
lup_con =vstack([lup_ceers,lup_gds,lup_gdn,lup_ngdeep,lup_udss,lup_udsn,lup_cosmos_east,lup_cosmos_west,lup_abell2744clu])


In [ ]:
#most small fields not in specz confirmed so not included rn 

lup_con_small = vstack([lup_abells1063,lup_elgordo,lup_j112716p4228,lup_macs0647,lup_rxj2129,lup_smacs0723,lup_sunrise])

In [ ]:
lup_total = vstack([lup_con,lup_con_small])

In [ ]:
#needs to be done in CP Lab 
lup_total.write('luptitudes_all_jwst_new.fits', format='fits', overwrite=True)

In [ ]:
t = Table.read('luptitudes_all_jwst_new.fits', format='fits')
t.add_column(np.nan, name='z_spec',index =4)


In [ ]:
print(len(t))
new_id = np.arange(0,len(t),1)
print(new_id)
t['id']= new_id

In [ ]:
t.write('luptitudes_all_jwst.fits',format = 'fits', overwrite=True)

# EXCELS Specz


In [ ]:
a = Table.read('redshift_cat_250124_coords.fits',format='fits')
flag = np.zeros(len(a))
for i in range(len(a)):
    if a['zflag_excels_acc'][i]==3  or a['zflag_excels_acc'][i]==4 or a['zflag_excels_acc'][i]==9:
        flag[i]=1
flag_b=np.array(flag,dtype=bool)

In [ ]:
print(np.max(a['z_excels_acc']))


# Skymatching Catalogues

### JWST match to CANDELS

In [3]:
candels=Table.read('MPhys/CANDELS/candels_combined.fits',format='fits')
jwst_merge=Table.read('MPhys/original_jwst_data/all_flux_merged.fits',format = 'fits')



In [4]:
flag_candels = candels['FLAGS']
jwst_merge.add_column(np.nan, name='z_spec',index =4)

In [5]:

flag_candels = candels['FLAGS']

f_arr= np.zeros(len(flag_candels))
for i in range(len(flag_candels)):

    if candels['FLAGS'][i]==0:
        f_arr[i]=1
f_bool =np.array(f_arr,dtype=bool)
##print(f_bool)
candels= candels[f_bool]
#candels

In [6]:
#matching jwst and candels
ra1=jwst_merge['ra']

dec1=jwst_merge['dec']
ra2=candels['RA']
dec2=candels['DEC']
print(len(ra1))

530526


In [7]:
jwst_cat = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree)#dawn
candels_cat = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree)#candels


In [8]:
max_sep = 0.1 * u.arcsec
idx, d2d, d3d = jwst_cat.match_to_catalog_sky(candels_cat)#catalog = candels information
idx_jwst, d2d_jwst, d3d_jwst = candels_cat.match_to_catalog_sky(jwst_cat)
#idx, d2d, d3d = jwst_cat.match_to_catalog_sky(candels_cat)
print(len(idx_jwst))

171218


In [11]:
print(len(np.unique(idx_jwst)))

103457


In [ ]:
c_index=[]
j_index=[]
for i in range(len(candels)):
    if abs(jwst_merge['ra'][idx[i]]- candels['RA'][i])<0.1:
        jwst_merge['z_spec'][idx_jwst[i]]=candels['z_spec'][i]


### JWST match to DAWN

In [ ]:
dawn = Table.read('csv.csv',format='csv')
ra3 = jwst_merge['ra']
dec3=jwst_merge['dec']
ra4=dawn['ra']
dec4=dawn['dec']
j_merge_cat = SkyCoord(ra=ra3*u.degree, dec=dec3*u.degree)
dawn_cat = SkyCoord(ra=ra4*u.degree, dec=dec4*u.degree)

In [ ]:
idx_all, d2d_all, d3d_all = dawn_cat.match_to_catalog_sky(j_merge_cat)

In [ ]:
c_index=[]
j_index=[]
for i in range(len(dawn)):
    if abs(jwst_merge['ra'][idx_all[i]]- dawn['ra'][i])<0.1:
        jwst_merge['z_spec'][idx_all[i]]=dawn['zspec'][i]


In [ ]:
jwst_merge['z_spec'][jwst_merge['z_spec']<=0]=np.ma.masked


z_dawn = dawn['zspec']
d = np.ones(len(z_dawn))
for i in range(len(dawn)):
    if np.ma.is_masked(z_dawn[i]):
        d[i]=0
    elif z_dawn[i]<=0:
        d[i]=0
d_bool = np.array(d,dtype=bool)        
dawn_good=dawn[d_bool]
dawn_high=dawn_good[dawn_good['zspec']>10]
dawn_high.show_in_notebook()

z_tst=jwst_merge['z_spec']
z_high =jwst_merge[z_tst>10]
z_high.show_in_notebook()

### JWST match to EXCELS

In [ ]:
a = Table.read('redshift_cat_250124_coords.fits',format='fits')
flag = np.zeros(len(a))
for i in range(len(a)):
    if a['zflag_excels_acc'][i]==3  or a['zflag_excels_acc'][i]==4 or a['zflag_excels_acc'][i]==9:
        flag[i]=1
flag_b=np.array(flag,dtype=bool)
a_good = a[flag_b]
a_good.sort('ra')
a_good

In [ ]:
a_good['z_excels_acc'][a_good['z_excels_acc'] <= 0] = np.nan

In [ ]:
ra5 = jwst_merge['ra']
dec5=jwst_merge['dec']
ra6=a_good['ra']
dec6=a_good['dec']
j_merge_cat2 = SkyCoord(ra=ra5*u.degree, dec=dec5*u.degree)
a_cat = SkyCoord(ra=ra6*u.degree, dec=dec6*u.degree)

In [ ]:
idx_a, d2d_a, d3d_a = a_cat.match_to_catalog_sky(j_merge_cat2)
print(jwst_merge['ra'][idx_a[0]]- a_good['ra'][0])
print(abs(ra6[0]-ra5[idx_all[0]]))

In [ ]:

for i in range(len(a_good)):
    if abs(jwst_merge['ra'][idx_a[i]]- a_good['ra'][i])<0.2:
       # print(i)
   #if abs(ra6[i]-ra5[idx_all[i]])<0.2:
       #'' print(i)
       # c_index.append(i)
       # j_index.append(idx_jwst[i])
        jwst_merge['z_spec'][idx_a[i]]=a_good['z_excels_acc'][i]


In [ ]:
jwst_merge['z_spec'][jwst_merge['z_spec']<=0]=np.ma.masked

In [ ]:
jwst_merge['id']=np.arange(0,len(jwst_merge),1)
jwst_merge

In [ ]:
s1 = jwst_merge[0:100000]
s2 = jwst_merge[100000:200000]
s3 = jwst_merge[200000:300000]
s4 = jwst_merge[300000:400000]
s5 = jwst_merge[400000:len(jwst_merge)]

In [ ]:
s1.write('MPhys/original_jwst_data/ab_final_skymatched_catalogue_excel_inc_s1.fits',format = 'fits',overwrite=True)

In [ ]:
s2.write('MPhys/original_jwst_data/ab_final_skymatched_catalogue_excel_inc_s2.fits',format = 'fits',overwrite=True)

In [ ]:
s3.write('MPhys/original_jwst_data/ab_final_skymatched_catalogue_excel_inc_s3.fits',format = 'fits',overwrite=True)

In [ ]:
s4.write('MPhys/original_jwst_data/ab_final_skymatched_catalogue_excel_inc_s4.fits',format = 'fits',overwrite=True)

In [ ]:
s5.write('MPhys/original_jwst_data/ab_final_skymatched_catalogue_excel_inc_s5.fits',format = 'fits',overwrite=True)

In [2]:
specz=Table.read('MPhys/original_jwst_data/final_skymatched_catalogue_excel_inc.fits',format='fits')

In [3]:
primer_specz = np.zeros(len(specz))
for i in range(len(specz)):
    if specz['field'][i]=='abell2744clu' or specz['field'][i]=='gdn' or specz['field'][i]=='gdn':
        primer_specz[i]=1
subset = specz[np.array(primer_specz, dtype = bool)]

In [4]:
subset

field,id,ra,dec,z_spec,lupt_f1000w_aper0,lupterr_f1000w_aper0,lupt_f105w_aper0,lupterr_f105w_aper0,lupt_f115w-clear_aper0,lupterr_f115w-clear_aper0,lupt_f125w_aper0,lupterr_f125w_aper0,lupt_f1280w_aper0,lupterr_f1280w_aper0,lupt_f140w_aper0,lupterr_f140w_aper0,lupt_f1500w_aper0,lupterr_f1500w_aper0,lupt_f150w-clear_aper0,lupterr_f150w-clear_aper0,lupt_f160w_aper0,lupterr_f160w_aper0,lupt_f1800w_aper0,lupterr_f1800w_aper0,lupt_f182m-clear_aper0,lupterr_f182m-clear_aper0,lupt_f200w-clear_aper0,lupterr_f200w-clear_aper0,lupt_f2100w_aper0,lupterr_f2100w_aper0,lupt_f210m-clear_aper0,lupterr_f210m-clear_aper0,lupt_f275wu_aper0,lupterr_f275wu_aper0,lupt_f277w-clear_aper0,lupterr_f277w-clear_aper0,lupt_f356w-clear_aper0,lupterr_f356w-clear_aper0,lupt_f410m-clear_aper0,lupterr_f410m-clear_aper0,lupt_f435w_aper0,lupterr_f435w_aper0,lupt_f444w-clear_aper0,lupterr_f444w-clear_aper0,lupt_f560w_aper0,lupterr_f560w_aper0,lupt_f606w_aper0,lupterr_f606w_aper0,lupt_f606wu_aper0,lupterr_f606wu_aper0,lupt_f770w_aper0,lupterr_f770w_aper0,lupt_f814w_aper0,lupterr_f814w_aper0,lupt_f1000w_aper1,lupterr_f1000w_aper1,lupt_f105w_aper1,lupterr_f105w_aper1,lupt_f115w-clear_aper1,lupterr_f115w-clear_aper1,lupt_f125w_aper1,lupterr_f125w_aper1,lupt_f1280w_aper1,lupterr_f1280w_aper1,lupt_f140w_aper1,lupterr_f140w_aper1,lupt_f1500w_aper1,lupterr_f1500w_aper1,lupt_f150w-clear_aper1,lupterr_f150w-clear_aper1,lupt_f160w_aper1,lupterr_f160w_aper1,lupt_f1800w_aper1,lupterr_f1800w_aper1,lupt_f182m-clear_aper1,lupterr_f182m-clear_aper1,lupt_f200w-clear_aper1,lupterr_f200w-clear_aper1,lupt_f2100w_aper1,lupterr_f2100w_aper1,lupt_f210m-clear_aper1,lupterr_f210m-clear_aper1,lupt_f275wu_aper1,lupterr_f275wu_aper1,lupt_f277w-clear_aper1,lupterr_f277w-clear_aper1,lupt_f356w-clear_aper1,lupterr_f356w-clear_aper1,lupt_f410m-clear_aper1,lupterr_f410m-clear_aper1,lupt_f435w_aper1,lupterr_f435w_aper1,lupt_f444w-clear_aper1,lupterr_f444w-clear_aper1,lupt_f560w_aper1,lupterr_f560w_aper1,lupt_f606w_aper1,lupterr_f606w_aper1,lupt_f606wu_aper1,lupterr_f606wu_aper1,lupt_f770w_aper1,lupterr_f770w_aper1,lupt_f814w_aper1,lupterr_f814w_aper1,lupt_f1000w_aper2,lupterr_f1000w_aper2,lupt_f105w_aper2,lupterr_f105w_aper2,lupt_f115w-clear_aper2,lupterr_f115w-clear_aper2,lupt_f125w_aper2,lupterr_f125w_aper2,lupt_f1280w_aper2,lupterr_f1280w_aper2,lupt_f140w_aper2,lupterr_f140w_aper2,lupt_f1500w_aper2,lupterr_f1500w_aper2,lupt_f150w-clear_aper2,lupterr_f150w-clear_aper2,lupt_f160w_aper2,lupterr_f160w_aper2,lupt_f1800w_aper2,lupterr_f1800w_aper2,lupt_f182m-clear_aper2,lupterr_f182m-clear_aper2,lupt_f200w-clear_aper2,lupterr_f200w-clear_aper2,lupt_f2100w_aper2,lupterr_f2100w_aper2,lupt_f210m-clear_aper2,lupterr_f210m-clear_aper2,lupt_f275wu_aper2,lupterr_f275wu_aper2,lupt_f277w-clear_aper2,lupterr_f277w-clear_aper2,lupt_f356w-clear_aper2,lupterr_f356w-clear_aper2,lupt_f410m-clear_aper2,lupterr_f410m-clear_aper2,lupt_f435w_aper2,lupterr_f435w_aper2,lupt_f444w-clear_aper2,lupterr_f444w-clear_aper2,lupt_f560w_aper2,lupterr_f560w_aper2,lupt_f606w_aper2,lupterr_f606w_aper2,lupt_f606wu_aper2,lupterr_f606wu_aper2,lupt_f770w_aper2,lupterr_f770w_aper2,lupt_f814w_aper2,lupterr_f814w_aper2,lupt_f1000w_aper3,lupterr_f1000w_aper3,lupt_f105w_aper3,lupterr_f105w_aper3,lupt_f115w-clear_aper3,lupterr_f115w-clear_aper3,lupt_f125w_aper3,lupterr_f125w_aper3,lupt_f1280w_aper3,lupterr_f1280w_aper3,lupt_f140w_aper3,lupterr_f140w_aper3,lupt_f1500w_aper3,lupterr_f1500w_aper3,lupt_f150w-clear_aper3,lupterr_f150w-clear_aper3,lupt_f160w_aper3,lupterr_f160w_aper3,lupt_f1800w_aper3,lupterr_f1800w_aper3,lupt_f182m-clear_aper3,lupterr_f182m-clear_aper3,lupt_f200w-clear_aper3,lupterr_f200w-clear_aper3,lupt_f2100w_aper3,lupterr_f2100w_aper3,lupt_f210m-clear_aper3,lupterr_f210m-clear_aper3,lupt_f275wu_aper3,lupterr_f275wu_aper3,lupt_f277w-clear_aper3,lupterr_f277w-clear_aper3,lupt_f356w-clear_aper3,lupterr_f356w-clear_aper3,lupt_f410m-clear_aper3,lupterr_f410m-clear_aper3,lupt_f435w_aper3,lupterr_f435w_aper3,lupt_f444w-clear_aper3,lu

In [7]:
primer.write('MPhys/original_jwst_data/primer_only.fits',format='fits')

In [8]:
primer = Table.read('MPhys/primer_excels_skymatched.fits',format='fits')

In [10]:
primer_good_spec = np.ones(len(primer))
for i in range(len(primer)):
    if np.ma.is_masked(primer['z_spec'][i]):
        primer_good_spec[i]=0
primer = primer[np.array(primer_good_spec, dtype = bool)]

In [15]:
primer.write('MPhys/primer_cosmos_skymatched_good_spec.fits',format='fits')

# Filter Choices for Training Catalogue


In [5]:
#train = Table.read('MPhys/catalogues/lupt_jwst_skymatched_specz.fits',format = 'fits')
#train = Table.read('MPhys/original_jwst_data/final_skymatched_abmag_catalogue.fits',format = 'fits')
train = Table.read('MPhys/original_jwst_data/final_skymatched_catalogue_excel_inc.fits',format = 'fits')

In [6]:
#train['z_spec'][train['z_spec']<0]=np.ma.masked
#t = Table.read('MPhys/original_jwst_data/final_skymatched_catalogue.fits',format = 'fits')

In [2]:
#train=Table.read('MPhys/original_jwst_data/final_skymatched_catalogue_excel_inc.fits',format='fits')


In [7]:
train

field,id,ra,dec,z_spec,lupt_f1000w_aper0,lupterr_f1000w_aper0,lupt_f105w_aper0,lupterr_f105w_aper0,lupt_f115w-clear_aper0,lupterr_f115w-clear_aper0,lupt_f125w_aper0,lupterr_f125w_aper0,lupt_f1280w_aper0,lupterr_f1280w_aper0,lupt_f140w_aper0,lupterr_f140w_aper0,lupt_f1500w_aper0,lupterr_f1500w_aper0,lupt_f150w-clear_aper0,lupterr_f150w-clear_aper0,lupt_f160w_aper0,lupterr_f160w_aper0,lupt_f1800w_aper0,lupterr_f1800w_aper0,lupt_f182m-clear_aper0,lupterr_f182m-clear_aper0,lupt_f200w-clear_aper0,lupterr_f200w-clear_aper0,lupt_f2100w_aper0,lupterr_f2100w_aper0,lupt_f210m-clear_aper0,lupterr_f210m-clear_aper0,lupt_f275wu_aper0,lupterr_f275wu_aper0,lupt_f277w-clear_aper0,lupterr_f277w-clear_aper0,lupt_f356w-clear_aper0,lupterr_f356w-clear_aper0,lupt_f410m-clear_aper0,lupterr_f410m-clear_aper0,lupt_f435w_aper0,lupterr_f435w_aper0,lupt_f444w-clear_aper0,lupterr_f444w-clear_aper0,lupt_f560w_aper0,lupterr_f560w_aper0,lupt_f606w_aper0,lupterr_f606w_aper0,lupt_f606wu_aper0,lupterr_f606wu_aper0,lupt_f770w_aper0,lupterr_f770w_aper0,lupt_f814w_aper0,lupterr_f814w_aper0,lupt_f1000w_aper1,lupterr_f1000w_aper1,lupt_f105w_aper1,lupterr_f105w_aper1,lupt_f115w-clear_aper1,lupterr_f115w-clear_aper1,lupt_f125w_aper1,lupterr_f125w_aper1,lupt_f1280w_aper1,lupterr_f1280w_aper1,lupt_f140w_aper1,lupterr_f140w_aper1,lupt_f1500w_aper1,lupterr_f1500w_aper1,lupt_f150w-clear_aper1,lupterr_f150w-clear_aper1,lupt_f160w_aper1,lupterr_f160w_aper1,lupt_f1800w_aper1,lupterr_f1800w_aper1,lupt_f182m-clear_aper1,lupterr_f182m-clear_aper1,lupt_f200w-clear_aper1,lupterr_f200w-clear_aper1,lupt_f2100w_aper1,lupterr_f2100w_aper1,lupt_f210m-clear_aper1,lupterr_f210m-clear_aper1,lupt_f275wu_aper1,lupterr_f275wu_aper1,lupt_f277w-clear_aper1,lupterr_f277w-clear_aper1,lupt_f356w-clear_aper1,lupterr_f356w-clear_aper1,lupt_f410m-clear_aper1,lupterr_f410m-clear_aper1,lupt_f435w_aper1,lupterr_f435w_aper1,lupt_f444w-clear_aper1,lupterr_f444w-clear_aper1,lupt_f560w_aper1,lupterr_f560w_aper1,lupt_f606w_aper1,lupterr_f606w_aper1,lupt_f606wu_aper1,lupterr_f606wu_aper1,lupt_f770w_aper1,lupterr_f770w_aper1,lupt_f814w_aper1,lupterr_f814w_aper1,lupt_f1000w_aper2,lupterr_f1000w_aper2,lupt_f105w_aper2,lupterr_f105w_aper2,lupt_f115w-clear_aper2,lupterr_f115w-clear_aper2,lupt_f125w_aper2,lupterr_f125w_aper2,lupt_f1280w_aper2,lupterr_f1280w_aper2,lupt_f140w_aper2,lupterr_f140w_aper2,lupt_f1500w_aper2,lupterr_f1500w_aper2,lupt_f150w-clear_aper2,lupterr_f150w-clear_aper2,lupt_f160w_aper2,lupterr_f160w_aper2,lupt_f1800w_aper2,lupterr_f1800w_aper2,lupt_f182m-clear_aper2,lupterr_f182m-clear_aper2,lupt_f200w-clear_aper2,lupterr_f200w-clear_aper2,lupt_f2100w_aper2,lupterr_f2100w_aper2,lupt_f210m-clear_aper2,lupterr_f210m-clear_aper2,lupt_f275wu_aper2,lupterr_f275wu_aper2,lupt_f277w-clear_aper2,lupterr_f277w-clear_aper2,lupt_f356w-clear_aper2,lupterr_f356w-clear_aper2,lupt_f410m-clear_aper2,lupterr_f410m-clear_aper2,lupt_f435w_aper2,lupterr_f435w_aper2,lupt_f444w-clear_aper2,lupterr_f444w-clear_aper2,lupt_f560w_aper2,lupterr_f560w_aper2,lupt_f606w_aper2,lupterr_f606w_aper2,lupt_f606wu_aper2,lupterr_f606wu_aper2,lupt_f770w_aper2,lupterr_f770w_aper2,lupt_f814w_aper2,lupterr_f814w_aper2,lupt_f1000w_aper3,lupterr_f1000w_aper3,lupt_f105w_aper3,lupterr_f105w_aper3,lupt_f115w-clear_aper3,lupterr_f115w-clear_aper3,lupt_f125w_aper3,lupterr_f125w_aper3,lupt_f1280w_aper3,lupterr_f1280w_aper3,lupt_f140w_aper3,lupterr_f140w_aper3,lupt_f1500w_aper3,lupterr_f1500w_aper3,lupt_f150w-clear_aper3,lupterr_f150w-clear_aper3,lupt_f160w_aper3,lupterr_f160w_aper3,lupt_f1800w_aper3,lupterr_f1800w_aper3,lupt_f182m-clear_aper3,lupterr_f182m-clear_aper3,lupt_f200w-clear_aper3,lupterr_f200w-clear_aper3,lupt_f2100w_aper3,lupterr_f2100w_aper3,lupt_f210m-clear_aper3,lupterr_f210m-clear_aper3,lupt_f275wu_aper3,lupterr_f275wu_aper3,lupt_f277w-clear_aper3,lupterr_f277w-clear_aper3,lupt_f356w-clear_aper3,lupterr_f356w-clear_aper3,lupt_f410m-clear_aper3,lupterr_f410m-clear_aper3,lupt_f435w_aper3,lupterr_f435w_aper3,lupt_f444w-clear_aper3,lu

In [8]:
new = np.linspace(int(0),int(len(train)-1),int(len(train)),dtype=int)
train['id']=new
train

field,id,ra,dec,z_spec,lupt_f1000w_aper0,lupterr_f1000w_aper0,lupt_f105w_aper0,lupterr_f105w_aper0,lupt_f115w-clear_aper0,lupterr_f115w-clear_aper0,lupt_f125w_aper0,lupterr_f125w_aper0,lupt_f1280w_aper0,lupterr_f1280w_aper0,lupt_f140w_aper0,lupterr_f140w_aper0,lupt_f1500w_aper0,lupterr_f1500w_aper0,lupt_f150w-clear_aper0,lupterr_f150w-clear_aper0,lupt_f160w_aper0,lupterr_f160w_aper0,lupt_f1800w_aper0,lupterr_f1800w_aper0,lupt_f182m-clear_aper0,lupterr_f182m-clear_aper0,lupt_f200w-clear_aper0,lupterr_f200w-clear_aper0,lupt_f2100w_aper0,lupterr_f2100w_aper0,lupt_f210m-clear_aper0,lupterr_f210m-clear_aper0,lupt_f275wu_aper0,lupterr_f275wu_aper0,lupt_f277w-clear_aper0,lupterr_f277w-clear_aper0,lupt_f356w-clear_aper0,lupterr_f356w-clear_aper0,lupt_f410m-clear_aper0,lupterr_f410m-clear_aper0,lupt_f435w_aper0,lupterr_f435w_aper0,lupt_f444w-clear_aper0,lupterr_f444w-clear_aper0,lupt_f560w_aper0,lupterr_f560w_aper0,lupt_f606w_aper0,lupterr_f606w_aper0,lupt_f606wu_aper0,lupterr_f606wu_aper0,lupt_f770w_aper0,lupterr_f770w_aper0,lupt_f814w_aper0,lupterr_f814w_aper0,lupt_f1000w_aper1,lupterr_f1000w_aper1,lupt_f105w_aper1,lupterr_f105w_aper1,lupt_f115w-clear_aper1,lupterr_f115w-clear_aper1,lupt_f125w_aper1,lupterr_f125w_aper1,lupt_f1280w_aper1,lupterr_f1280w_aper1,lupt_f140w_aper1,lupterr_f140w_aper1,lupt_f1500w_aper1,lupterr_f1500w_aper1,lupt_f150w-clear_aper1,lupterr_f150w-clear_aper1,lupt_f160w_aper1,lupterr_f160w_aper1,lupt_f1800w_aper1,lupterr_f1800w_aper1,lupt_f182m-clear_aper1,lupterr_f182m-clear_aper1,lupt_f200w-clear_aper1,lupterr_f200w-clear_aper1,lupt_f2100w_aper1,lupterr_f2100w_aper1,lupt_f210m-clear_aper1,lupterr_f210m-clear_aper1,lupt_f275wu_aper1,lupterr_f275wu_aper1,lupt_f277w-clear_aper1,lupterr_f277w-clear_aper1,lupt_f356w-clear_aper1,lupterr_f356w-clear_aper1,lupt_f410m-clear_aper1,lupterr_f410m-clear_aper1,lupt_f435w_aper1,lupterr_f435w_aper1,lupt_f444w-clear_aper1,lupterr_f444w-clear_aper1,lupt_f560w_aper1,lupterr_f560w_aper1,lupt_f606w_aper1,lupterr_f606w_aper1,lupt_f606wu_aper1,lupterr_f606wu_aper1,lupt_f770w_aper1,lupterr_f770w_aper1,lupt_f814w_aper1,lupterr_f814w_aper1,lupt_f1000w_aper2,lupterr_f1000w_aper2,lupt_f105w_aper2,lupterr_f105w_aper2,lupt_f115w-clear_aper2,lupterr_f115w-clear_aper2,lupt_f125w_aper2,lupterr_f125w_aper2,lupt_f1280w_aper2,lupterr_f1280w_aper2,lupt_f140w_aper2,lupterr_f140w_aper2,lupt_f1500w_aper2,lupterr_f1500w_aper2,lupt_f150w-clear_aper2,lupterr_f150w-clear_aper2,lupt_f160w_aper2,lupterr_f160w_aper2,lupt_f1800w_aper2,lupterr_f1800w_aper2,lupt_f182m-clear_aper2,lupterr_f182m-clear_aper2,lupt_f200w-clear_aper2,lupterr_f200w-clear_aper2,lupt_f2100w_aper2,lupterr_f2100w_aper2,lupt_f210m-clear_aper2,lupterr_f210m-clear_aper2,lupt_f275wu_aper2,lupterr_f275wu_aper2,lupt_f277w-clear_aper2,lupterr_f277w-clear_aper2,lupt_f356w-clear_aper2,lupterr_f356w-clear_aper2,lupt_f410m-clear_aper2,lupterr_f410m-clear_aper2,lupt_f435w_aper2,lupterr_f435w_aper2,lupt_f444w-clear_aper2,lupterr_f444w-clear_aper2,lupt_f560w_aper2,lupterr_f560w_aper2,lupt_f606w_aper2,lupterr_f606w_aper2,lupt_f606wu_aper2,lupterr_f606wu_aper2,lupt_f770w_aper2,lupterr_f770w_aper2,lupt_f814w_aper2,lupterr_f814w_aper2,lupt_f1000w_aper3,lupterr_f1000w_aper3,lupt_f105w_aper3,lupterr_f105w_aper3,lupt_f115w-clear_aper3,lupterr_f115w-clear_aper3,lupt_f125w_aper3,lupterr_f125w_aper3,lupt_f1280w_aper3,lupterr_f1280w_aper3,lupt_f140w_aper3,lupterr_f140w_aper3,lupt_f1500w_aper3,lupterr_f1500w_aper3,lupt_f150w-clear_aper3,lupterr_f150w-clear_aper3,lupt_f160w_aper3,lupterr_f160w_aper3,lupt_f1800w_aper3,lupterr_f1800w_aper3,lupt_f182m-clear_aper3,lupterr_f182m-clear_aper3,lupt_f200w-clear_aper3,lupterr_f200w-clear_aper3,lupt_f2100w_aper3,lupterr_f2100w_aper3,lupt_f210m-clear_aper3,lupterr_f210m-clear_aper3,lupt_f275wu_aper3,lupterr_f275wu_aper3,lupt_f277w-clear_aper3,lupterr_f277w-clear_aper3,lupt_f356w-clear_aper3,lupterr_f356w-clear_aper3,lupt_f410m-clear_aper3,lupterr_f410m-clear_aper3,lupt_f435w_aper3,lupterr_f435w_aper3,lupt_f444w-clear_aper3,lu

In [ ]:
#Full catalogue filter names 
filters_1 = [col.split('_')[1] for col in train.colnames if col.endswith('_aper1')]
#print(len(filters_1))
print(train.colnames)

In [24]:
#### FILTER CHOICES
bc= ['f115w-clear','f150w-clear','f356w-clear','f444w-clear']
#swap f115w for f814w,f606w,f435w,f125w,f140w,f160w,f150w
c1 =['f814w','f200w-clear','f356w-clear','f444w-clear']
c2 =['f606w','f200w-clear','f356w-clear','f444w-clear']
c3 =['f435w','f200w-clear','f356w-clear','f444w-clear']
c4 =['f125w','f200w-clear','f356w-clear','f444w-clear']
c5 =['f140w','f200w-clear','f356w-clear','f444w-clear']
c6 =['f160w','f200w-clear','f356w-clear','f444w-clear']
c7 =['f150w-clear','f200w-clear','f356w-clear','f444w-clear']

#swap f444w for f410m
c9= ['f115w-clear','f200w-clear','f356w-clear','f410m-clear']
c10=['f115w-clear','f606w','f150w-clear','f444w-clear']
bc_extra = ['f090w-clear','f115w-clear','f200w-clear','f356w-clear','f444w-clear']
swap= ['bc','f814w','f606w','f435w','f125w','f140w','f160w','f150w-clear','f410m-clear','ic','f090w_inc']
#num = 0

choice=bc
num = 0

print(swap[num])
lup_train = train['field','id','ra','dec','z_spec','flux_auto','flux_aper_1']
lup_train[f'lupt_{choice[0]}']=train[f'{choice[0]}_aper_{aper}']
lup_train[f'lupterr_{choice[0]}']=train[f'{choice[0]}_aper_{aper}']
lup_train[f'lupt_{choice[1]}']=train[f'{choice[1]}_aper_{aper}']
lup_train[f'lupterr_{choice[1]}']=train[f'{choice[1]}_aper_{aper}']
lup_train[f'lupt_{choice[2]}']=train[f'{choice[2]}_aper_{aper}']
lup_train[f'lupterr_{choice[2]}']=train[f'{choice[2]}_aper_{aper}']
lup_train[f'lupt_{choice[3]}']=train[f'{choice[3]}_aper_{aper}']
lup_train[f'lupterr_{choice[3]}']=train[f'{choice[3]}_aper_{aper}']
lup_train

In [25]:
######change this to make new catalogue of filters
aper=1
aper_size=2
train =subset
choice=bc
num = 0

print(swap[num])
lup_train = train['field','id','ra','dec','z_spec']
lup_train[f'lupt_{choice[0]}']=train[f'lupt_{choice[0]}_aper{aper}']
lup_train[f'lupterr_{choice[0]}']=train[f'lupterr_{choice[0]}_aper{aper}']
lup_train[f'lupt_{choice[0]}_aper2']=train[f'lupt_{choice[0]}_aper{aper_size}']
lup_train[f'lupterr_{choice[0]}_aper2']=train[f'lupterr_{choice[0]}_aper{aper_size}']

lup_train[f'lupt_{choice[1]}']=train[f'lupt_{choice[1]}_aper{aper}']
lup_train[f'lupterr_{choice[1]}']=train[f'lupterr_{choice[1]}_aper{aper}']#
lup_train[f'lupt_{choice[1]}_aper2']=train[f'lupt_{choice[1]}_aper{aper_size}']
lup_train[f'lupterr_{choice[1]}_aper2']=train[f'lupterr_{choice[1]}_aper{aper_size}']

lup_train[f'lupt_{choice[2]}']=train[f'lupt_{choice[2]}_aper{aper}']
lup_train[f'lupterr_{choice[2]}']=train[f'lupterr_{choice[2]}_aper{aper}']
lup_train[f'lupt_{choice[2]}_aper2']=train[f'lupt_{choice[2]}_aper{aper_size}']
lup_train[f'lupterr_{choice[2]}_aper2']=train[f'lupterr_{choice[2]}_aper{aper_size}']

lup_train[f'lupt_{choice[3]}']=train[f'lupt_{choice[3]}_aper{aper}']
lup_train[f'lupterr_{choice[3]}']=train[f'lupterr_{choice[3]}_aper{aper}']
lup_train[f'lupt_{choice[3]}_aper2']=train[f'lupt_{choice[3]}_aper{aper_size}']
lup_train[f'lupterr_{choice[3]}_aper2']=train[f'lupterr_{choice[3]}_aper{aper_size}']

#lup_train[f'lupt_{choice[4]}']=train[f'lupt_{choice[4]}_aper{aper}']
#lup_train[f'lupterr_{choice[4]}']=train[f'lupterr_{choice[4]}_aper{aper}']
#lup_train[f'lupt_{choice[4]}_aper2']=train[f'lupt_{choice[4]}_aper{aper_size}']
#lup_train[f'lupterr_{choice[4]}_aper2']=train[f'lupterr_{choice[4]}_aper{aper_size}']
#Adds size column 
#lup_train[f'lupt_{choice[4]}_size']=train[f'lupt_{choice[4]}_aper{aper_size}']/train[f'lupt_{choice[4]}_aper{aper}']
#lup_train[f'lupterr_{choice[4]}_size']=train[f'lupterr_{choice[4]}_aper{aper_size}']/train[f'lupterr_{choice[4]}_aper{aper}']

bc


In [26]:
lup_train

field,id,ra,dec,z_spec,lupt_f115w-clear,lupterr_f115w-clear,lupt_f115w-clear_aper2,lupterr_f115w-clear_aper2,lupt_f150w-clear,lupterr_f150w-clear,lupt_f150w-clear_aper2,lupterr_f150w-clear_aper2,lupt_f356w-clear,lupterr_f356w-clear,lupt_f356w-clear_aper2,lupterr_f356w-clear_aper2,lupt_f444w-clear,lupterr_f444w-clear,lupt_f444w-clear_aper2,lupterr_f444w-clear_aper2
bytes13,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
gdn,128727,189.38940935133246,62.15017499961043,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
gdn,128728,189.23176682893654,62.15030632505291,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
gdn,128729,189.20338663060625,62.15031093470169,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
gdn,128730,189.18719032458648,62.15031079171647,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
gdn,128731,189.13862480980308,62.15029995401969,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
gdn,128732,189.12878618922161,62.15029539106479,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
gdn,128733,189.0734073626092,62.15025777934166,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
gdn,128734,189.0470650513054,62.150232308846924,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
gdn,128735,189.0077500367946,62.15018481747126,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--


In [43]:
f1=[]
for i in range(len(lup_train)):
    f1.append(np.ma.is_masked(lup_train[f'lupt_{choice[0]}'][i]))
f1_inv=np.array(np.invert(f1))
f1_cat =lup_train[f1_inv*(np.ma.is_masked(lup_train[f'lupt_{choice[0]}']) == True)] 

f2=[]
for i in range(len(f1_cat)):
    f2.append(np.ma.is_masked(f1_cat[f'lupt_{choice[1]}'][i]))
f2_inv=np.array(np.invert(f2))
f2_cat =f1_cat[f2_inv*(np.ma.is_masked(f1_cat[f'lupt_{choice[1]}']) ==True)] 

f3=[]
for i in range(len(f2_cat)):
    f3.append(np.ma.is_masked(f2_cat[f'lupt_{choice[2]}'][i]))
f3_inv=np.array(np.invert(f3))
f3_cat =f2_cat[f3_inv*(np.ma.is_masked(f2_cat[f'lupt_{choice[2]}']) == True)] 


f4=[]
for i in range(len(f3_cat)):
    f4.append(np.ma.is_masked(f3_cat[f'lupt_{choice[3]}'][i]))
f4_inv=np.array(np.invert(f4))
f4_cat =f3_cat[f4_inv*(np.ma.is_masked(f3_cat[f'lupt_{choice[3]}']) == False)] #false idk why but works 

f4_cat.write(f'MPhys/gmm_files/{swap[num]}_eazy_comp_subset_dawn_only.fits',format='fits',overwrite=True)
#f5=[]
#for i in range(len(f4_cat)):
#    f5.append(np.ma.is_masked(f4_cat[f'lupt_{choice[4]}'][i]))
#f5_inv=np.array(np.invert(f5))
#f5_cat =f4_cat[f5_inv*(np.ma.is_masked(f4_cat[f'lupterr_{choice[4]}']) ==True)] 

In [44]:
#USE IF USING 5TH FILTER
#f5=[]
#for i in range(len(f4_cat)):
#    f5.append(np.ma.is_masked(f4_cat[f'lupt_{choice[4]}'][i]))
#f5_inv=np.array(np.invert(f5))
#f5_cat =f4_cat[f5_inv*(np.ma.is_masked(f4_cat[f'lupterr_{choice[4]}']) ==True)] 

In [46]:
f4_cat

field,id,ra,dec,z_spec,lupt_f115w-clear,lupterr_f115w-clear,lupt_f115w-clear_aper2,lupterr_f115w-clear_aper2,lupt_f150w-clear,lupterr_f150w-clear,lupt_f150w-clear_aper2,lupterr_f150w-clear_aper2,lupt_f356w-clear,lupterr_f356w-clear,lupt_f356w-clear_aper2,lupterr_f356w-clear_aper2,lupt_f444w-clear,lupterr_f444w-clear,lupt_f444w-clear_aper2,lupterr_f444w-clear_aper2
bytes13,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
gdn,128736,189.36052340425985,62.150222033713845,--,27.868484522676543,0.1520875330629353,28.001351078766017,0.21786650819351036,28.83291099500146,0.33037863364162756,28.64198738441131,0.3710063167445948,28.144668364639237,0.11305375616017319,28.13269231897249,0.1453641365079515,28.793656177508428,0.22331343522277475,28.56117414390061,0.22911926900469862
gdn,128737,189.35898435007283,62.15022029088431,--,29.10809627389637,0.35175758857364364,28.694973916568195,0.32770374365082655,28.132779131063156,0.16470107613383544,27.859595936277326,0.17050092536493044,28.1224860289991,0.11071468941466078,27.821508860414482,0.1129298496484816,28.70374069384031,0.21404556162494687,28.479836016468585,0.2249289395500827
gdn,128738,189.349278721561,62.150236398048136,--,27.966775700059557,0.16710376046283193,27.8014496550665,0.18838777167415743,27.689571494507646,0.11518749590127025,27.47877206810305,0.12574412811307636,28.153889536146956,0.11796248731265445,27.949111399653148,0.1318237263993997,27.592397904430037,0.1646925741231714,27.21471890874327,0.16753418351259794
gdn,128739,189.377417083452,62.150206511511946,--,27.49528935373428,0.21668565673657125,27.46621676108997,0.28071442378527534,26.698203804917842,0.0734709052754246,26.517822280613096,0.08124423243908528,26.297890163802688,0.08841466572818148,26.067846224734172,0.08068413538697991,26.37502227316324,0.19034251438999025,26.112786146446588,0.1618217278311133
gdn,128740,189.30710771155861,62.15029389518875,--,28.432410773732396,0.3044288520535671,28.063117135430097,0.2875741918514208,27.89721348532353,0.16322699748000494,27.708684513643256,0.1788675369476508,28.222975719432043,0.13631012699878006,28.335126995536324,0.19179481840055476,28.458372901171668,0.22221383100455044,28.49727490718173,0.2594801528353791
gdn,128741,189.32933564325438,62.150260681984435,--,25.51135825636543,0.02165367994200799,25.39210419804598,0.025259653912104645,25.6531429553607,0.020366163261823816,25.506948924715285,0.023316679620872246,25.48158514207069,0.014399829372069234,25.353537188356473,0.017069563811306428,25.380991614366597,0.013901382095045729,25.246705569978506,0.015996357004879187
gdn,128742,189.31336916643914,62.15032708258601,--,27.825907028155296,0.20180024744645772,27.444601980922712,0.19624381910284547,28.68548339737459,0.3321475756071791,28.12087122123364,0.2847684419532031,28.39431103873646,0.16704136436175596,28.16246920357069,0.1864259561275849,28.53952555941016,0.2318516498049244,28.258047333304017,0.23806722088478927
gdn,128743,189.31036421342804,62.150274880065005,--,27.688041659972853,0.1579234319243333,27.40217888889466,0.1600066999601992,27.62326685203475,0.12072949251972576,27.4350300533235,0.13292855728194752,27.41407973746895,0.06171092905493828,27.145533228567462,0.06670271443219038,27.451106461196016,0.0814957178863417,27.15432465184439,0.08433900266421133
gdn,128744,189.30821665735584,62.15034450623081,--,28.504375847243665,0.33347704528387906,28.563252714168776,0.43695459407557746,28.119471133531764,0.20595093827942124,27.909271947974435,0.234852428498105,27.54174986356276,0.07748486445468342,27.383756110035144,0.09400277802114108,27.672139943593674,0.11174638621563228,27.384326532779212,0.11889389299732651


In [47]:
f4_cat.write(f'MPhys/gmm_files/{swap[num]}_extra_apertures_full_catalogue_dawn_only.fits',format='fits',overwrite=True)
#f5_cat.write(f'MPhys/base_filter_permutations/base_filter_permutations/size/informed_choice_size.cat',format = 'ascii.commented_header',overwrite=True)

In [48]:
#f5_cat['z_spec'][f5_cat['z_spec']==0]=np.nan

In [52]:
#MAKE THE TRAINING CATALOGUE

training=[]
for i in range(len(f4_cat)):
    training.append(np.ma.is_masked(f4_cat['z_spec'][i]))
training_inv=np.array(np.invert(training))
training_cat =f4_cat[training_inv*(np.ma.is_masked(f4_cat['z_spec']) == True)] 

In [53]:
training_cat

field,id,ra,dec,z_spec,lupt_f115w-clear,lupterr_f115w-clear,lupt_f115w-clear_aper2,lupterr_f115w-clear_aper2,lupt_f150w-clear,lupterr_f150w-clear,lupt_f150w-clear_aper2,lupterr_f150w-clear_aper2,lupt_f356w-clear,lupterr_f356w-clear,lupt_f356w-clear_aper2,lupterr_f356w-clear_aper2,lupt_f444w-clear,lupterr_f444w-clear,lupt_f444w-clear_aper2,lupterr_f444w-clear_aper2
bytes13,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
gdn,129376,189.31358977342478,62.15840214221569,0.5609,21.182212900477374,0.0008176532907320832,20.95265106878457,0.000792196956351871,20.799530025834365,0.0006361337915852021,20.57717378938419,0.0006017099998119801,20.850219711388906,0.0006205840145907985,20.60256693388318,0.0005670887210838471,21.282898957982578,0.0007984532470742888,21.010303414190727,0.0007368226418309802
gdn,129445,189.3033081748749,62.15939028537185,0.8206,22.8001102992208,0.002413513335526961,22.51963390475459,0.002442645108012131,22.11594671123716,0.0013728490634171532,21.85151962818224,0.0013418019962698302,21.00898575700349,0.0006680665625251928,20.738247404851087,0.0006027228389365234,20.993004085756855,0.000682555789319344,20.696564692667817,0.0006149983491114596
gdn,129486,189.3034406068834,62.16026172364166,0.4807,27.646343739581628,0.19739006526588074,27.310799195890652,0.1977388079470189,27.69621806583862,0.14878069553498877,27.319134691215304,0.14608569674192567,27.319135942959747,0.06595994006582666,26.995620391368462,0.06792327407267552,27.57149111089721,0.106655982399149,27.215045064795174,0.10675572764662486
gdn,129489,189.32384058316754,62.16048392460884,0.229,23.969818919481302,0.005989389803869911,23.796938537890227,0.006922753227398029,23.914621197934217,0.00511861576858459,23.74271966614722,0.005813497358331883,24.73092500731812,0.008030323508821413,24.521415315162226,0.008852053913199664,25.005901751575497,0.015683338821075702,24.798755950730218,0.017902432546342075
gdn,129506,189.3054370624384,62.16075246754981,1.3977,24.06964207506408,0.006847522494164729,23.952206360131385,0.008242180069830666,23.372390780767677,0.003319154393601237,23.26976687855285,0.00392418518521829,22.479418954143192,0.0014553134486662038,22.3642780865659,0.001509227053394087,22.393131050347996,0.001507079194510138,22.240274319676516,0.0015767032816404658
gdn,129934,189.33267657758455,62.16531854997747,2.647,24.089889637787234,0.008363986726624677,24.01320660231067,0.010478468935342057,23.33725359145394,0.003222886004114621,23.24543860462143,0.0038231395813225267,21.922599037903918,0.0010686483446728176,21.817279360485802,0.0010815774639652582,21.3750930786205,0.0008429799666704996,21.235879972806533,0.0008373931858843915
gdn,129943,189.32750414954415,62.16550282012746,2.268,24.423920796439212,0.008898334726237455,23.919726916664303,0.007743027777968184,23.919205009376604,0.005066500785477921,23.42866232954605,0.004464621455266019,24.168647846793647,0.0045472273333472795,23.706521921748394,0.00395284104978147,24.14399845211859,0.005359878215078825,23.69235734841312,0.004756315492655813
gdn,129973,189.31424873929785,62.165572825825556,0.3919,23.595899715259133,0.004460441374006946,23.106116572910604,0.003906845814785077,23.4654671908628,0.003758486851983759,22.978743605079455,0.003195382483418342,24.025876765180328,0.004053816260158786,23.521739174378656,0.0034287279330042116,24.246087789606072,0.006631969537529081,23.73253471888011,0.0057515915411472905
gdn,129980,189.32982353788876,62.16541010975437,0.7779,22.711740460890166,0.0022089899710538877,22.443637153378475,0.00223355726142605,22.305103618802328,0.001552403121949559,22.06703128871576,0.0015564216675459286,21.908856479448783,0.0010817992035458922,21.6561686046801,0.001027659746198228,22.334181142507195,0.0014517901333145305,22.0618867367647,0.0013988854854701952


In [54]:
training_cat.write(f'MPhys/gpz_catalogues/{swap[num]}_eazy_comp_subset_training_dawn_only.cat',format='ascii.commented_header',overwrite=True)

# PRIMER Catalogue for GMM 
### Base Filters only initially 

In [ ]:
full = Table.read('MPhys/gmm_files/bc_luptitude_full_catalogue_new.fits',format = 'fits')
train_gmm = Table.read('MPhys/gpz_catalogues/bc_full_training_cat_new.cat',format = 'ascii.commented_header')

In [ ]:
pc_train = np.zeros(len(train_gmm))
for i in range(len(train_gmm)):
    if train_gmm['field'][i]== 'cosmos_east' or train_gmm['field'][i]=='cosmos_west':
        pc_train[i]=1

pc_train_bool = np.array(pc_train,dtype = bool)
train_gmm_pc = train_gmm[pc_train_bool]

In [ ]:
train_gmm_pc.write('MPhys/gmm_files/primer_only_train.fits',format = 'fits',overwrite=True)

In [ ]:
pc = np.zeros(len(full))
for i in range(len(full)):
    if full['field'][i]== 'cosmos_east' or full['field'][i]=='cosmos_west':
        pc[i]=1
pc_bool = np.array(pc,dtype = bool)
full_pc = full[pc_bool]

In [ ]:
full_pc.write('MPhys/gmm_files/primer_only_full.fits',format = 'fits',overwrite=True)

# Split into high and low redshift


In [ ]:

t_full =Table.read(f'MPhys/gpz_catalogues/{swap[num]}_full_training_cat_size.cat',format='ascii.commented_header')
#t_full.sort('z_spec')
z= t_full['z_spec']
bins = np.linspace(np.min(z),np.max(z),5)
h2 = np.zeros(len(z))
h3 = np.zeros(len(z))
h4 = np.zeros(len(z))
low = np.zeros(len(z))

for i in range(len(z)):
    if bins[0]<=z[i]<=bins[1]:
        low[i]=1
    elif bins[1]<=z[i]<=bins[2]:
        h2[i]=1
    elif bins[2]<=z[i]<=bins[3]:
        h3[i]=1
    elif bins[3]<=z[i]<=bins[4]:
        h4[i]=1
h2_bool = np.array(h2, dtype =bool)
h3_bool = np.array(h3, dtype =bool)
h4_bool = np.array(h4, dtype =bool)
low_bool = np.array(low, dtype =bool)

low_cat = t_full[low_bool]
h2_cat = t_full[h2_bool]
h3_cat = t_full[h3_bool]
h4_cat = t_full[h4_bool]
h2_lim = int(len(h2_cat)*0.8)
h3_lim = int(len(h3_cat)*0.8)
h4_lim = int(len(h4_cat)*0.8)
low_lim = int(len(low_cat)*0.8)

t_low = low_cat[0:low_lim]
p_low = low_cat[low_lim:len(low_cat)]

t_h2 = h2_cat[0:h2_lim]
p_h2 = h2_cat[h2_lim:len(h2_cat)]
t_h3 = h3_cat[0:h3_lim]
p_h3 = h3_cat[h3_lim:len(h3_cat)]
t_h4 = h4_cat[0:h4_lim]
p_h4 = h4_cat[h4_lim:len(h4_cat)]


t_high = vstack([t_h2,t_h3,t_h4])
p_high = vstack([p_h2,p_h3,p_h4])


In [ ]:
t_all = vstack([t_low,t_high])
p_all = vstack([p_low,p_high])


In [ ]:
t_low.write(f'MPhys/gpz_catalogues/gpz_catalogues/reg_z_split/lowz/TRAIN/train_lowz_{swap[num]}.cat',format = 'ascii.commented_header',overwrite=True)
p_low.write(f'MPhys/gpz_catalogues/gpz_catalogues/reg_z_split/lowz/PRED/pred_lowz_{swap[num]}.cat',format = 'ascii.commented_header',overwrite=True)
t_high.write(f'MPhys/gpz_catalogues/gpz_catalogues/reg_z_split/highz/TRAIN/train_highz_{swap[num]}.cat',format = 'ascii.commented_header',overwrite=True)
p_high.write(f'MPhys/gpz_catalogues/gpz_catalogues/reg_z_split/highz/PRED/pred_highz_{swap[num]}.cat',format = 'ascii.commented_header',overwrite=True)

t_all.write(f'MPhys/gpz_catalogues/gpz_catalogues/reg/TRAIN/train_{swap[num]}.cat',format = 'ascii.commented_header',overwrite=True)
p_all.write(f'MPhys/gpz_catalogues/gpz_catalogues/reg/PRED/pred_{swap[num]}.cat',format = 'ascii.commented_header',overwrite=True)